Suguru Maker/analisys/solver/grapher

Suguru is a sudoku like game where you solve a n*p grid filled by 1 - k size zones containing number from 1 to the zone size. 

a number cannot be touching (meaning in a 3*3 grid centered on a number, that number cannot be present again)

In [80]:
import numpy as np
import operator
import random

Suguru Maker

In [81]:
def is_valid(index : tuple, shape : tuple):

    return (0 <= index[0] < shape[0] and
            0 <= index[1] < shape[1])

In [82]:
def get_neighboors(position :tuple, shape : tuple):
    neighboors = []
    # adds position in neighboors remove how ? 
    for i in range(-1, 2):
        for j in range(-1, 2):
            if is_valid((position[0]+i,position[1]+j),shape):
                neighboors.append((position[0]+i,position[1]+j))
    neighboors.remove(position)
    return neighboors


In [84]:
def get_positions_snail(positions :np.array , grid : np.array, shape :tuple):
    directions = [np.array([0,1]),np.array([1,0]),np.array([0,-1]),np.array([-1,0])]
    direction = 0
    old_pos = positions[0]
    positions[1] = old_pos + directions[direction]
    pos = positions[1]
    grid[*old_pos] = 1
    grid[*pos] = 1
    i=2
    
    while i < len(positions):
        
        if is_valid(pos + directions[(direction+1)%4],shape):
        # i.e can be seen as if the snail can coil more ( that is_valid is techniacally always true)
            if grid[tuple(pos + directions[(direction+1)%4])] == 0:
                pos = pos + directions[(direction+1)%4]
                positions[i] = pos
                grid[*pos] = 1
                direction = (direction+1)%4
                i += 1
            
            elif is_valid(pos + directions[direction],shape):
                if grid[tuple(pos + directions[direction])] == 0:
                    pos = pos + directions[direction]
                    positions[i] = pos
                    grid[*pos] = 1
                    i += 1

                else:
                    # goes over previous values ( to see the scenario just imagine a snail path that starts with an offset from the center)
                    pos = pos + directions[direction]
            else:
                # turns at a dead end ( also to see the scenario just imagine a snail path that starts with an offset from the center)
                direction = (direction+1)%4
    return positions, grid
                

                
            




In [85]:
def get_neighboor_line(prev :tuple , grid : np.array, shape :tuple):
    # lots of optimizations possible here just general architecture useless calculus and verif but who cares
    # goes right then down 
    val = np.copy(prev)
    val[1] +=1 
    print("val",val)

    if is_valid(val,shape):
        #checks if this lands in the suguru grid

        if not(grid[*val]):
            #checks if there is a value assigned there
            return val
    else:
        val[1] = 0

        if not(grid[*val]):
            return val
    val[1] = prev[1]
    val[0] += 1 
    if is_valid(val,shape):
        if not(grid[*val]):
            return val
    else:
        val[0] = 0
        if not(grid[*val]):
            return val
    print("ERROR")
    return prev
    




In [130]:
# def get_value(suguru: np.array, neighboor:tuple):
#     return
    

In [131]:
def avaliable(value : int, numbers : list):
    return True

    if value == 1:
        return True
    else:
        if value-1 in numbers:
            return(avaliable(value-1, numbers))

In [127]:
def get_possibilities(numbers : list, counter : list):
    #check the values of the neighboors to know which numbers can be added

    ###NOT MATHEMATICALLY 
    
    #we can add a check if all the numbers under present exist in a 3d radius
    
    possibilities = []
    for value in range(1,6):
        if not (value in numbers):
            if value != 1 :
                if avaliable(value, numbers):
                    possibilities.append(value)                    
            else:
                possibilities.append(value)
    for value in range(2,6):

        if counter[value-1] >= counter[value-2]:
            if value in possibilities:
                possibilities.remove(value)
    return possibilities


In [89]:
def get_value(possibilities : np.array):
    #we can try equiprob since we only add a 5 in poss if a 4 is present
    return random.choice(possibilities)        
        


In [90]:
def path_type(method):
    methods = ["line" ,"snail"]
    if method in methods:
        return method
    else : 
        print("unnamed method switching to line")
        return "line"

In [128]:
def make_numbers(suguru : np.array):
    
    #writes the numbers first hopefully there wont be impossible grids? 
    grid_shape = np.shape(suguru)
    grid_size = grid_shape[0]*grid_shape[1]
    #useless for now ? 
    # average_zone_size = 4
    # zone_number = np.ceil(np.size(suguru)/average_zone_size)

    # pick a random spot throw a 1
    # adding a -1 here doesnt change a thing factually just makes it easier to encode snail for now
    # slows very very very slightly by adding 2 operations in the whole calculus  
    start_x = np.random.randint(grid_shape[0]-1)
    start_y = np.random.randint(grid_shape[1]-1)

    suguru[start_x][start_y] = 1

    grid = np.zeros(grid_shape)
    #grid gets allocated positions
    # allocation_type = "line"
    # technique = "line"
    technique = "snail"
    technique = path_type(technique)
    i = 0
    position = (start_x, start_y)
    positions = np.zeros((64,2)).astype(int)
    positions[0] = position

    if technique == "line":
        for i in range (1,64):
            positions[i] = (get_neighboor_line(positions[i-1], grid, grid_shape)) 
            grid[*positions[i]] = 1
    if technique == "snail" : 
        positions, grid = get_positions_snail(positions, grid, grid_shape)
        

    i = 0
    counter = [0,0,0,0,0]

    while i<grid_size:
        
        # position = get_neighboor_line(position,grid, grid_shape)

        #pick a neighboor
        #assess possible values (if nothing possible here i-e 1-5 is taken then what?) what if snail shape allocation ? should work since no more than 4 neighbors max ? 
        #can do snail-like allocation from random startr point technically #most arranged allocation techniques should work, while random based ones are unsafe
        #distribution will be weighted to have more 1s in the firstly elected part likely ?
        position = tuple(positions[i])
        neighboors = get_neighboors(position, grid_shape)
        numbers = []
        for neighboor in neighboors:
            numbers.append(int(suguru[neighboor]))
        possibilities = get_possibilities(numbers, counter)
        value = get_value(possibilities)
        counter[value-1] += 1
        print("counter",counter )
        suguru[position] = value
        grid[position] = 1

    #allocate values
    
        i+=1

    print(suguru)
    return suguru

    
    

In [ ]:
def make_suguru(size = 8):
    # makes a Filled Suguru grid 
    if 2 < size < 14 : 
        # suguru = np.zeros((size,size)).astype(int)
        suguru = make_numbers(suguru)
    else:
         suguru = np.zeros
    return suguru

In [ ]:
suguru = np.zeros((size,size)).astype(int)

suguru = make_suguru(8)

counter [1, 0, 0, 0, 0]
counter [1, 1, 0, 0, 0]
counter [1, 1, 1, 0, 0]
counter [1, 1, 1, 1, 0]
counter [1, 1, 1, 1, 1]


IndexError: Cannot choose from an empty sequence

In [115]:
#numbers of each val in suguru
print(suguru)
for i in range(1,6):
    print(f"number of {i}s : {np.sum(suguru==i)}")
    

[[1 3 4 1 4 3 5 2]
 [5 2 5 3 5 2 1 4]
 [4 1 4 2 1 4 5 3]
 [5 3 5 3 5 3 1 2]
 [4 2 4 2 4 2 4 3]
 [3 1 3 5 1 3 5 1]
 [4 2 4 2 4 2 4 2]
 [3 1 3 1 3 1 3 1]]
number of 1s : 13
number of 2s : 12
number of 3s : 15
number of 4s : 14
number of 5s : 10
